In [ ]:
# did you remember to change the 'device' in general_framework.py ?
from tutorialQA_framework import *
from tutorial1_framework import *
# add more here, or comment out

device = torch.device('cuda:1') # let's use the alligator GPU
device

pygame 2.6.1 (SDL 2.28.4, Python 3.12.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
🔥 text_pretraining_data/eng_sentences_pruned-train.txt


In [ ]:
from visual_transformer import *
from visual_transformer.enhanced_model import *

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(G.getData())

In [ ]:
def quick_display(torch_img):
    clean = torch_img.detach().cpu()
    right_order = torch.permute(clean, (1, 2, 0))
    array = right_order.numpy()
    plt.imshow(array)

In [ ]:
brain = EnhancedAgentBrain()
brain.move_to(device) # special function that wraps 'to'. Dumb? yes. Needed? Also yes.

brain.load_state_dict(torch.load('brain_checkpoints/enhanced_brain_first_training_batch10000.pth', weights_only=True, map_location=device))

In [ ]:
gen_optimizer = optim.Adam(brain.parameters(), lr=0.00001, eps=1e-9)

In [ ]:
# should possibly also include mem_enc? Should just be gen_optimizer? 
text_optimizer = optim.Adam(list(brain.text_enc.parameters()) + list(brain.text_dec.parameters()), lr=0.00001, eps=1e-9)

In [ ]:
# useful to randomize the order in which the tasks are trained
class ReusableBuffer:
    def __init__(self, L):
        self.L = L
        self.inds = list(range(len(L)))

    def draw(self, ind):
        return self.L[ind]

    def random_draw(self):
        ind_ind = random.randint(0, len(self.inds)-1)
        ind = self.inds[ind_ind]
        if ind_ind == (len(self.inds) - 1):
            self.inds = self.inds[:-1]
        else:
            self.inds = self.inds[:ind_ind] + self.inds[ind_ind + 1:]
        if len(self.inds) == 0:
            self.inds = list(range(len(self.L)))
        return self.L[ind], ind

In [ ]:
# for first task (and really anywhere I want to not reset between tasks)
# make sure the batch size matches
rb = ReusableBuffer([(arrow_task_batch, gen_optimizer, 16), \
                     (qa_task_batch, text_optimizer, 16)]) # add further functions here

In [ ]:
batches = [0, 0]

In [ ]:
curr_mins = [1000.0, 1000.0]

In [ ]:
total_losses = [0, 0]

In [ ]:
#torch.autograd.set_detect_anomaly(True)

In [ ]:
total_batches = 10000*100#6250*32

for b in range(total_batches):
    triplet, ind = rb.random_draw()
    func, opt, batch_size = triplet
    
    batch_num = batches[ind]
    batches[ind] += 1

    #reset_model = True #default option; only transfer memory within the task files
    reset_model = (b % 3 == 2)

    printing = ((batch_num % 100) == 99)
    full_results = func(batch_size, brain, optimizer=opt, batch_num=batch_num, compute_grad=True, random_order=True, model_eval=False, reset_model=reset_model, printing=printing, training=True)
    L = full_results[0] # no need to look into the detailed loss report
    total_losses[ind] += L

    if printing: # if this is a significant batch
        avg_loss = total_losses[ind] / 100
        total_losses[ind] = 0
        print(f"Average total loss for task {ind}, last 100 batches: {avg_loss}\n")
        
        if avg_loss < curr_mins[ind]:
            curr_mins[ind] = avg_loss
            torch.save(brain.state_dict(), f"brain_checkpoints/enhanced_brain_first_training_v2_batch{b + 1}.pth")
            
    if b < 10:
        print(f"batch {b}, task {ind}, task batch_num {batch_num}\n")

In [ ]:
# Conclusion: not even close on the results. Need several days (basically seems to be relearning task1; taskQA basically
# completed)

In [ ]:
# Rerunning on top of the last results, with many more batches (so several days)